# Meshes

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from ngsolve import *
import matplotlib.pyplot as plt
import sys
import gc

sys.path.insert(1, "/Users/gpin/Repos/EVErrorEst/scripts")

from main_utils import *
from adapt_lshape import *
from error import *
from ngsolve.webgui import Draw
from IPython.display import display, HTML

In [2]:
def reset_mesh():
    return Mesh(make_l_shape().GenerateMesh(maxh=0.3))


def error_estimator_landscape(gf):
    """
    Compute the landscape error estimator.
    """
    h = specialcf.mesh_size
    n = specialcf.normal(gf.space.mesh.dim)
    xs = [x, y, z][: gf.space.mesh.dim]

    grad_gf = grad(gf)
    div_grad_gf = sum((grad_gf[i].Diff(xs[i]) for i in range(gf.space.mesh.dim)))

    integrand_1 = -div_grad_gf - 1.0
    integrand_2 = (grad_gf - grad_gf.Other()) * n

    eta_1 = Integrate(
        h**2 * InnerProduct(integrand_1, integrand_1) * dx,
        gf.space.mesh,
        element_wise=True,
    )
    eta_2 = Integrate(
        0.5 * h * InnerProduct(integrand_2, integrand_2) * dx(element_boundary=True),
        gf.space.mesh,
        element_wise=True,
    )

    eta = np.sqrt(eta_1.NumPy().real + eta_2.NumPy().real)
    etas = {
        "eta_1": eta_1.NumPy().real,
        "eta_2": eta_2.NumPy().real,
    }

    return eta, etas


def error_estimator_ev(gf, ev):
    """
    Compute the landscape error estimator.
    """
    h = specialcf.mesh_size
    n = specialcf.normal(gf.space.mesh.dim)
    xs = [x, y, z][: gf.space.mesh.dim]

    grad_gf = grad(gf)
    div_grad_gf = sum((grad_gf[i].Diff(xs[i]) for i in range(gf.space.mesh.dim)))

    integrand_1 = -div_grad_gf - ev * gf
    integrand_2 = (grad_gf - grad_gf.Other()) * n

    eta_1 = Integrate(
        h**2 * InnerProduct(integrand_1, integrand_1) * dx,
        gf.space.mesh,
        element_wise=True,
    )
    eta_2 = Integrate(
        0.5 * h * InnerProduct(integrand_2, integrand_2) * dx(element_boundary=True),
        gf.space.mesh,
        element_wise=True,
    )

    eta = np.sqrt(eta_1.NumPy().real + eta_2.NumPy().real)
    etas = {
        "eta_1": eta_1.NumPy().real,
        "eta_2": eta_2.NumPy().real,
    }

    return eta, etas

## Mesh Comparison

In [3]:
# FES parameters
ORDER = 1
IS_COMPLEX = True

# Adaptivity parameters
MAX_ITER = 10
MAX_NDOFS = 300_000
# THETA = 0.95

# FEAST parameters
NSPAN = 6
CHECKS = False
RADIUS = 2.0

### Landscape refinement

In [4]:
mesh = reset_mesh()
fes = H1(mesh, order=ORDER, complex=IS_COMPLEX, dirichlet="boundary", autoupdate=True)

u, v = fes.TnT()

a = BilinearForm(fes)
a += grad(u) * grad(v) * dx

m = BilinearForm(fes)
m += u * v * dx

f = LinearForm(fes)
f += 1.0 * v * dx

assemble(a, m, f)

#### High tolerance

In [5]:
THETA = 0.95

iteration = 0
ndofs = 0
sol = GridFunction(fes, name="Landscape", autoupdate=True)

# Main loop
Draw(mesh)
while iteration < MAX_ITER and ndofs < MAX_NDOFS:
    # Update
    assemble(a, f)
    ndofs = fes.ndof
    iteration += 1
    # Solve
    sol.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
    # Estimate
    eta, _ = error_estimator_landscape(sol)
    # Mark
    max_eta = max(eta)
    mesh.ngmesh.Elements2D().NumPy()["refine"] = eta > THETA * max_eta
    # Refine
    mesh.Refine()
    Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

...

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

#### Low tolerance

In [6]:
mesh = reset_mesh()
fes = H1(mesh, order=ORDER, complex=IS_COMPLEX, dirichlet="boundary", autoupdate=True)

u, v = fes.TnT()

a = BilinearForm(fes)
a += grad(u) * grad(v) * dx

m = BilinearForm(fes)
m += u * v * dx

f = LinearForm(fes)
f += 1.0 * v * dx

assemble(a, m, f)

In [7]:
THETA = 0.5

iteration = 0
ndofs = 0
sol = GridFunction(fes, name="Landscape", autoupdate=True)

# Main loop
Draw(mesh)
while iteration < MAX_ITER and ndofs < MAX_NDOFS:
    # Update
    assemble(a, f)
    ndofs = fes.ndof
    iteration += 1
    # Solve
    sol.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
    # Estimate
    eta, _ = error_estimator_landscape(sol)
    # Mark
    max_eta = max(eta)
    mesh.ngmesh.Elements2D().NumPy()["refine"] = eta > THETA * max_eta
    # Refine
    mesh.Refine()
    Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

...

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

### Eigenvector-driven refinement

#### Low tolerance

In [8]:
REF_EV = 50 * np.pi**2

feast_parameters = {
    "hermitian": False,
    "stop_tol": 1.0e-10,
    "cut_tol": 1.0e-10,
    "eta_tol": 1e-10,
    "nrestarts": 5,
    "niterations": 100,
}

mesh = reset_mesh()
fes = H1(mesh, order=ORDER, complex=IS_COMPLEX, dirichlet="boundary", autoupdate=True)

u, v = fes.TnT()

a = BilinearForm(fes)
a += grad(u) * grad(v) * dx

m = BilinearForm(fes)
m += u * v * dx

f = LinearForm(fes)
f += 1.0 * v * dx

assemble(a, m, f)

In [9]:
THETA = 0.5

iteration = 0
ndofs = 0
center = REF_EV

sol = GridFunction(fes, name="Landscape", autoupdate=True)

# Main loop
while iteration < MAX_ITER and ndofs < MAX_NDOFS:
    # Update
    assemble(a, m, f)
    ndofs = fes.ndof
    iteration += 1
    # Solve
    sol.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
    # --- Init  call FEAST ---
    # Set clean eigenspace (required for adaptivity)
    right = NGvecs(fes, NSPAN)
    right.setrandom()
    left = NGvecs(fes, NSPAN)
    left.setrandom()
    # Set spectral projector
    projector = SpectralProjNG(
        fes,
        a.mat,
        m.mat,
        checks=CHECKS,
        radius=RADIUS,
        center=center,
        npts=NPTS,
        verbose=False,
    )
    # Call FEAST
    evalues, right, history, left = projector.feast(right, Yl=left, **feast_parameters)
    assert history[-1], "FEAST did not converge"
    # --- End  call  FEAST ---
    # Estimate
    etas = []
    for i in range(right.m):
        eta, _ = error_estimator_ev(right.gridfun(i=i), evalues[i])
        etas.append(eta)
    etas = np.stack(etas, axis=0)
    eta = np.max(etas, axis=0)
    # Mark
    max_eta = max(eta)
    mesh.ngmesh.Elements2D().NumPy()["refine"] = eta > THETA * max_eta
    # Refine
    mesh.Refine()
    Draw(mesh)
    # Clean up
    del right, left, projector, history
    gc.collect()

   QR found abs(diag(R)):
   [3.5e+00 3.5e+00 3.0e+00 2.1e+00 2.2e+00 1.7e+00 1.3e+00 2.4e+00 1.9e+00
 9.1e-01 8.9e-01 7.4e-01 1.4e+00 1.7e+00 1.9e+00 1.1e+00 1.2e+00 5.7e-01
 7.3e-01 1.3e+00 2.9e-01 7.5e-22 1.2e-21 4.9e-22]
   Removing 3 vector(s).
   QR found abs(diag(R)):
   [1.2e+01 1.1e+01 1.2e+01 7.7e+00 6.0e+00 8.0e+00 5.5e+00 6.0e+00 4.4e+00
 3.3e+00 2.4e+00 1.7e+00 3.3e+00 3.2e+00 4.1e+00 1.8e+00 1.9e+00 1.9e+00
 2.0e+00 1.9e+00 8.4e-01 2.6e-21 2.7e-21 8.4e-22]
   Removing 3 vector(s).


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

...

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

#### High tolerance

In [10]:
REF_EV = 50 * np.pi**2

feast_parameters = {
    "hermitian": False,
    "stop_tol": 1.0e-10,
    "cut_tol": 1.0e-10,
    "eta_tol": 1e-10,
    "nrestarts": 5,
    "niterations": 100,
}

mesh = reset_mesh()
fes = H1(mesh, order=ORDER, complex=IS_COMPLEX, dirichlet="boundary", autoupdate=True)

u, v = fes.TnT()

a = BilinearForm(fes)
a += grad(u) * grad(v) * dx

m = BilinearForm(fes)
m += u * v * dx

f = LinearForm(fes)
f += 1.0 * v * dx

assemble(a, m, f)

In [11]:
THETA = 0.95

iteration = 0
ndofs = 0
center = REF_EV

sol = GridFunction(fes, name="Landscape", autoupdate=True)

# Main loop
while iteration < MAX_ITER and ndofs < MAX_NDOFS:
    # Update
    assemble(a, m, f)
    ndofs = fes.ndof
    iteration += 1
    # Solve
    sol.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
    # --- Init  call FEAST ---
    # Set clean eigenspace (required for adaptivity)
    right = NGvecs(fes, NSPAN)
    right.setrandom()
    left = NGvecs(fes, NSPAN)
    left.setrandom()
    # Set spectral projector
    projector = SpectralProjNG(
        fes,
        a.mat,
        m.mat,
        checks=CHECKS,
        radius=RADIUS,
        center=center,
        npts=NPTS,
        verbose=False,
    )
    # Call FEAST
    evalues, right, history, left = projector.feast(right, Yl=left, **feast_parameters)
    assert history[-1], "FEAST did not converge"
    # --- End  call  FEAST ---
    # Estimate
    etas = []
    for i in range(right.m):
        eta, _ = error_estimator_ev(right.gridfun(i=i), evalues[i])
        etas.append(eta)
    etas = np.stack(etas, axis=0)
    eta = np.max(etas, axis=0)
    # Mark
    max_eta = max(eta)
    mesh.ngmesh.Elements2D().NumPy()["refine"] = eta > THETA * max_eta
    # Refine
    mesh.Refine()
    Draw(mesh)
    # Clean up
    del right, left, projector, history
    gc.collect()

   QR found abs(diag(R)):
   [1.0e+01 8.1e+00 6.1e+00 4.0e+00 3.5e+00 4.3e+00 4.4e+00 3.8e+00 3.2e+00
 2.6e+00 3.3e+00 2.0e+00 3.0e+00 2.7e+00 2.3e+00 2.4e+00 1.7e+00 1.4e+00
 2.2e+00 1.6e+00 2.0e+00 8.4e-22 7.1e-22 1.0e-21]
   Removing 3 vector(s).
   QR found abs(diag(R)):
   [4.2e+00 5.2e+00 6.3e+00 4.8e+00 5.1e+00 3.6e+00 4.0e+00 4.0e+00 3.8e+00
 3.2e+00 2.9e+00 1.8e+00 2.4e+00 3.1e+00 2.2e+00 2.2e+00 2.8e+00 1.7e+00
 2.1e+00 1.7e+00 1.3e+00 9.1e-22 1.5e-21 5.3e-22]
   Removing 3 vector(s).


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

   QR found abs(diag(R)):
   [1.4e+00 5.4e+02 5.2e+00 1.3e+00 1.6e+00 7.8e-01 5.0e-01 4.1e-01 1.8e-01
 6.3e-01 9.9e-02 1.4e-01 3.5e-02 1.7e-01 2.2e-01 2.8e-01 1.4e-01 1.7e-01
 1.8e-01 1.9e-01 1.3e-01 1.7e-01 2.2e-01 1.3e-01 1.5e-01 2.0e-01 3.6e-02
 2.2e-17 4.5e-17 9.2e-17 5.9e-17 2.4e-15 2.0e-16 1.4e-16 1.1e-16 2.6e-16
 6.6e-17 2.7e-17 2.3e-16 1.3e-16 4.1e-17 1.1e-16 1.3e-16 1.5e-16 7.4e-17
 1.2e-16 2.0e-16 6.4e-17]
   Removing 21 vector(s).
   QR found abs(diag(R)):
   [6.5e+01 2.1e-01 1.3e+01 2.0e+01 9.4e+00 1.3e+01 6.8e+00 9.4e+00 7.0e+00
 2.4e+01 1.4e+01 1.1e+01 7.1e+00 9.1e+00 1.5e+01 3.0e+01 8.9e+00 1.8e+01
 2.2e+01 2.0e+01 1.0e+01 4.8e+00 1.7e+01 4.7e+00 1.6e+00 5.9e+00 1.6e-04
 5.7e-16 1.2e-16 5.2e-16 1.8e-16 4.5e-15 3.8e-16 3.4e-16 3.2e-16 1.2e-16
 2.5e-16 9.8e-17 1.9e-16 1.1e-16 2.2e-16 2.4e-16 2.6e-16 2.5e-16 3.2e-16
 1.7e-16 1.1e-16 1.4e-16]
   Removing 21 vector(s).


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

...

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

### Uniform

In [12]:
mesh = reset_mesh()
fes = H1(mesh, order=ORDER, complex=IS_COMPLEX, dirichlet="boundary", autoupdate=True)

u, v = fes.TnT()

a = BilinearForm(fes)
a += grad(u) * grad(v) * dx

m = BilinearForm(fes)
m += u * v * dx

f = LinearForm(fes)
f += 1.0 * v * dx

assemble(a, m, f)

In [13]:
iteration = 0
ndofs = 0

sol = GridFunction(fes, name="Landscape", autoupdate=True)

# Main loop
while iteration < MAX_ITER and ndofs < MAX_NDOFS:
    # Update
    assemble(a, m, f)
    ndofs = fes.ndof
    iteration += 1
    # Solve
    sol.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
    # Refine
    mesh.Refine()
    Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…